In [1]:
import gymnasium as gym
from time import sleep, time
import sys
sys.path.insert(0,"/home/sipekd/Desktop/ReinforcementLearning")

print(sys.path)

from IPython.display import clear_output
from lib.TaxiEnv import TaxiEnv
from lib.utilities import renderStates

env = TaxiEnv()

def bfs(env, start):
    queue = [(start, [start])]
    while queue:
        state, path = queue.pop(0)
        for action in range(env.allActionCount):
            env.setState( state)
            new_state, reward, done, _, _ = env.step(action)

            if done:
                #yield path + [new_state]   # this would be to find all possible paths
                return path + [new_state]   # return the first one (the shortest one in case of BFS)
            elif new_state not in path:
                queue.append((new_state, path + [new_state]))


def dfs(env, start, maxLength = 20):
    # if(maxLength == None):
    #     maxLength = 
    deadWays = []
    shortest = float("inf")
    queue = [(start, 0, [start])]
    returnVal = []
    while queue:
        state, length, path = queue.pop()
        if(length > maxLength or length > shortest):
            deadWays.append(state)
            continue
        for action in range(env.allActionCount):
            env.setState(state)
            new_state, reward, done, _, _ = env.step(action)

            if done and shortest > length:
                shortest = length #Update "Max length to newer number" 
                returnVal = path + [new_state] #Make it a new chosen path
            elif new_state not in path:
                queue.append((new_state,length+1,path+[new_state]))
    return returnVal #After all the runs, return what we consider shortest

['/home/sipekd/Desktop/ReinforcementLearning', '/home/sipekd/Desktop/ReinforcementLearning/mbu', '/home/sipekd/anaconda3/envs/bp/lib/python312.zip', '/home/sipekd/anaconda3/envs/bp/lib/python3.12', '/home/sipekd/anaconda3/envs/bp/lib/python3.12/lib-dynload', '', '/home/sipekd/anaconda3/envs/bp/lib/python3.12/site-packages']


In [2]:
import pickle, os
import numpy as np

dfsTimes = []
bfsTimes = []
savename = "basic-search-algo-results.pickle"
 
if(not os.path.isfile(savename)):

    for StartState in range(500):

        t0 = time()
        shortest_path = dfs(env, start=StartState)
        t = time()-t0

        t01 = time()
        shortest_path2 = bfs(env, start=StartState)
        t1 = time()-t01
        dfsTimes.append(t)
        bfsTimes.append(t1)
        if(StartState%10==0):
            print(".",end="")
            if(StartState%100==0):
                print("/")


    saveThis = {"dfs" : dfsTimes, "bfs": bfsTimes}

    with open(savename,"wb") as file:
        pickle.dump(saveThis,file)
        file.close()

else:
    with open(savename,"rb") as file:
        x = pickle.load(file)
        dfsTimes = np.array(x["dfs"])
        bfsTimes = np.array(x["bfs"])

In [3]:
from BFS import bfs as bfs2
from DFS import dfs as dfs2
from lib.utilities import renderActions
from lib.utilities import renderStates
times1 = []
times2 = []
err_count = 0
errStates = []

otherPaths = []
others = 0

for StartState in range(500):
    
    # t0 = time()
    # shortest_path = dfs(env, start=328)
    # t = time()-t0

    # for state in shortest_path:
    #     env.setState(state)
    #     clear_output()
    #     print(env.render())
    #     sleep(0.42)

    #renderStates(shortest_path)

    t0 = time()
    shortest_path = dfs(env, start=StartState)
    t = time()-t0

    t0 = time()
    shortest_path2 = dfs2(env, start=StartState)
    t2 = time()-t0

    times1.append(t)
    times2.append(t2)

    if(shortest_path != shortest_path2):
        # print("target:" + shortest_path)
        # print("my:" + shortest_path2)
        if(len(shortest_path) != len(shortest_path2)):
            err_count+=1
            errStates.append([StartState,shortest_path,shortest_path2])
        else:
            others += 1
            otherPaths.append([StartState,shortest_path,shortest_path2])

    #print(shortest_path == shortest_path2)

    print(".", end= "")
    if((StartState+1) %100 == 0):
        print("")
print(err_count)
print(others)

/home/sipekd/anaconda3/envs/bp/lib/python3.12/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(
/home/sipekd/anaconda3/envs/bp/lib/python3.12/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.action_mask to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.action_mask` for environment variables or `env.get_wrapper_attr('action_mask')` that will search the reminding wrappers.
  logger.warn(


....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
0
359


In [4]:
StartState

499

In [5]:
errStates

[]

In [10]:
speedupTimes = []
minimum = float("inf")
maximum = float("-inf")
for i in range(len(times1)):
    speedupTimes.append(times1[i] / times2[i])

In [12]:
speedup = np.array(speedupTimes)
print("Minimum: " + str(speedup.min()) + "x at:"  + str(speedup.argmin()) )
print("Maximum: " + str(speedup.max()) + "x at:"  + str(speedup.argmax()) )
print("Průměr: "+ str(speedup.mean())+"x")


Minimum: 0.09039295997404599x at:499
Maximum: 4893.786697811654x at:218
Průměr: 127.70106746634212x


BFS: (Intel I7 1185G7 - 4C 8T @3.0GHz)   

Minimum: 0.6838445807770961x at:379   
Maximum: 365.6223128407429x at:4    
Průměr: 40.26918843553583x

DFS:
Minimum: 0.09039295997404599x at:499  
Maximum: 4893.786697811654x at:218  
Průměr: 127.70106746634212x  


In [8]:
renderStates(bfs2(env,4))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [9]:
### Merged to our implementation in previous cell
# t0 = time()
# shortest_path = bfs(env, start=328)
# t = time()-t0

# for state in shortest_path:
#     env.reset()
#     env.unwrapped.s = state
#     clear_output()
#     print(env.render())
#     sleep(0.42)

# print(f'Shortest path of len {len(shortest_path)} found in {t} seconds.')
# print(f'>> {shortest_path}')